
# Setup

In [ ]:
# Setup directories
from pathlib import Path
basedir = Path().absolute()
libdir = basedir.parent.parent.parent

# Other imports
import pandas as pd
import numpy as np
from datetime import datetime

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file, output_notebook
output_notebook()

In [ ]:
# Install QC library
!pip install git+git://github.com/ioos/ioos_qc.git

# # Alternative installation (install specific branch):
# !pip uninstall -y ioos_qc
# !pip install git+git://github.com/ioos/ioos_qc.git@BRANCHNAME

# # Alternative installation (run with local updates):
# !pip uninstall -y ioos_qc
# import sys
# sys.path.append(str(libdir))
    
from ioos_qc.config import QcConfig
from ioos_qc import qartod

In [ ]:
# Method to plot QC results using Bokeh
def plot_results(data, var_name, results, title, test_name):

    time = data['time']
    obs = data[var_name]
    qc_test = results['qartod'][test_name]

    qc_pass = np.ma.masked_where(qc_test != 1, obs)
    qc_suspect = np.ma.masked_where(qc_test != 3, obs)
    qc_fail = np.ma.masked_where(qc_test != 4, obs)
    qc_notrun = np.ma.masked_where(qc_test != 2, obs)

    p1 = figure(x_axis_type="datetime", title=test_name + ' : ' + title)
    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Time'
    p1.yaxis.axis_label = 'Observation Value'

    p1.line(time, obs,  legend_label='obs', color='#A6CEE3')
    p1.circle(time, qc_notrun, size=2, legend_label='qc not run', color='gray', alpha=0.2)
    p1.circle(time, qc_pass, size=4, legend_label='qc pass', color='green', alpha=0.5)
    p1.circle(time, qc_suspect, size=4, legend_label='qc suspect', color='orange', alpha=0.7)
    p1.circle(time, qc_fail, size=6, legend_label='qc fail', color='red', alpha=1.0)

    #output_file("qc.html", title="qc example")

    show(gridplot([[p1]], plot_width=800, plot_height=400))


# Specify data and QC configuration

In [ ]:
# Water level data
# For a fixed station in Kotzebue, AK (https://www.google.com/maps?q=66.895035,-162.566752)
filename = basedir.joinpath('water_level_example.csv')
variable_name='sea_surface_height_above_sea_level'

# QC configuration
# For sea water temperature in degrees C
# This configuration is used to call the corresponding method in the ioos_qc library
# See documentation for description of each test and its inputs: 
#   https://ioos.github.io/ioos_qc/api/ioos_qc.html#module-ioos_qc.qartod
qc_config = {
    'qartod': {
      "gross_range_test": {
        "fail_span": [-10,10],
        "suspect_span": [-2,3]
      },
      "flat_line_test": {
        "tolerance": 0.001,
        "suspect_threshold": 10800,
        "fail_threshold": 21600
      },
      "rate_of_change_test": {
        "threshold": 0.001
      },
      "spike_test": {
        "suspect_threshold": 0.8,
        "fail_threshold": 3
      },
      "aggregate": {}
    }
}

# Load data, run tests and plot results

In [ ]:
# Load data
data = pd.read_csv(filename, parse_dates=['time'])
data.head()

In [ ]:
# Run QC
qc = QcConfig(qc_config)
qc_results =  qc.run(
    inp=data[variable_name],
    tinp=data['timestamp'],
    zinp=data['z']
)
qc_results


In [ ]:
# Plot results
title = "Water Level [MHHW] [m] : Kotzebue, AK"

plot_results(data, variable_name, qc_results, title, 'gross_range_test')

In [ ]:
plot_results(data, variable_name, qc_results, title, 'flat_line_test')

In [ ]:
plot_results(data, variable_name, qc_results, title, 'rate_of_change_test')

In [ ]:
plot_results(data, variable_name, qc_results, title, 'spike_test')

In [ ]:
# QC Aggregate flag
plot_results(data, variable_name, qc_results, title, 'aggregate')